In [ ]:
import os
import json

from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.series import get_series_object

import csv
from library_dicom.dicom_processor.model.Series import Series

In [ ]:
series_paths = get_series_path('/home/deeplearning/AHL/AHL_Validated_DICOM')
export_folder = '/home/deeplearning/AHL/AHL_JSON'

In [ ]:
#Number of serie 
len(series_paths)

In [ ]:
index_problem = []
for serie_path in series_paths:
    print(series_paths.index(serie_path))
    try:
        dicom_serie = get_series_object(serie_path)
        dicomsInfo = dicom_serie.get_series_details()
        write_json_file(export_folder, dicomsInfo['series']['SeriesInstanceUID'], dicomsInfo)
    except Exception as err:
        print(err)
        print(dicomsInfo)
        index_problem.append(series_paths.index(serie_path))

In [ ]:
json_path = '/home/deeplearning/AHL/AHL_JSON'
list_dir_json = os.listdir(json_path)
list_json = []
for file_ in list_dir_json : 
    list_json.append([os.path.join(json_path, file_)])

In [ ]:
for liste in list_json : 
    print(list_json.index(liste))
    with open(liste[0]) as json_file : 
        reader = json.load(json_file)

        patient_id = reader["patient"]["PatientID"]
        study_uid = reader["study"]['StudyInstanceUID']
        modal = reader['series']['Modality']

        path = reader['path']

        type_ = reader['study']["AccessionNumber"]

        liste.append(path)
        liste.append(modal)
        liste.append(type_)
        liste.append(patient_id)
        liste.append(study_uid)

In [ ]:
#get every study_uid 
stud = []
for liste in list_json : 
    stud.append(liste[-1])

study_uid = []
study_uid.append(stud[0])
for uid in stud : 
    if uid not in study_uid : 
        study_uid.append(uid)

In [ ]:
print('Nombre de study :', len(study_uid))

In [ ]:
data = []
for uid in study_uid : 
    sub = []
    for liste in list_json : 
        if uid in liste : 
            sub.append(liste)
    data.append(sub)


In [ ]:
dataset = []
for serie in data : 
    serie[0].append(serie[1][0])
    serie[0].append(serie[1][1])
    serie[0].append(serie[1][2])

    dataset.append(serie[0])



In [ ]:
#Clean dataset : if no json, if double study uid, if not CT&PT 

#no json 

for data in dataset : 
    if len(data) != 9 : 
        print(data)

In [ ]:
#double study_uid 
double = []
double.append(dataset[0])
for data in dataset : 
    if data not in double : 
        double.append(data)

if len(double) == len(dataset) : 
    print('No double study uid')

In [ ]:
#check Modality 
index = []
for data in dataset : 
    modal = []
    modal.append(data[2])
    modal.append(data[-1])
    print(modal)
    if modal[0] == modal[1] : 
        index.append(dataset.index(data))
print(index)
#if index is not empty : 
if len(index) != 0 :
    liste_to_remove = []
    for ind in index : 
        liste_to_remove.append(dataset[ind])

    for r in liste_to_remove : 
        dataset.remove(r)

In [ ]:
#Now, can prepare json file to generate nifti in "nifti_builder_from_json.ipynb"

In [ ]:
study_results = []
for liste in dataset : 
    subliste = []
    subliste.append(liste[1]) #path_1
    subliste.append(liste[2]) #modal 1
    subliste.append(liste[-2]) #path_2
    subliste.append(liste[-1]) #modal_2
    subliste.append(liste[5]) #study_uid
    subliste.append(liste[3]) #type
    subliste.append(liste[4]) #patient id

    study_results.append(subliste)

In [ ]:
#save results as json 
directory = '/home/deeplearning/AHL'
filename = 'AHL_new_dataset'
write_json_file(directory, filename, study_results)